# Traitement des variables catégorielles
Pour pouvoir être passée à la plupart des modèles (globalement tous sauf les arbres de décision), une variable quelle qu'elle soit doit être de type numérique, un nombre. La plupart des modèles sous-entendent en effet que leur espace des *features* est $\mathbb{R}^{n}$. La présence de variables catégorielles pose la question du passage d'un ensemble de valeur discrètes à des valeurs dans un espace "continu" (pas forcément unidimensionnel). Ces techniques se regroupent sous le terme d'*encoding* qu'on distingue des plus sophistiqués *embeddings*.

Les aspects à étudier pour chaque variable catégorielle avant de commencer sont:
* **Le caractère ordinal** ou non, le premier appelant à des méthodes spécifiques.
* **La cardinalité** qui peut amener à écarter les méthodes les plus simples.
* **La distribution des effectifs** sur chacune des catégories qui peut exiger de prétraiter la variable en regroupant les effectifs des classes les plus faiblement peuplées (qui a aussi pour effet bénéfique de réduire la cardinalité).

Remarque sur le regroupement de classes: 
* Il peut prendre plusieurs formes, on peut simplement regrouper toutes les classes dans une classe "Other", passer à une maille moins fine (ex: pays > continent) ou bien sûr utiliser des logiques métier.
* Essayer dans la mesure du possible que les nouvelles classes soient relativement équilibrées en effectifs, ou à mimnima, que les effectifs dans les plus petites soient suffisant.
* C'est un paliatif simple aux problèmes de dimensionnalité engendrés par certains *encoders*. 

## *Encoders*
L'encoder le plus simple et le plus classique est le *one-hot encoding* qui introduit une variable binaire (*dummy variable*) pour chaque niveau de la variable catégorielle traitée (de $n$ niveaux, on crée $n-1$ nouvelles variables binaires). Bien qu'utile cette approche approche assez vite ses limites quand la cardinalité des variables à encoder est élevée : elle fait rapidement exploser la dimensionalité du problème, l'ensemble des nouvelles colonnes associées étant de plus très *sparse*. Au delà de la *curse of dimensionality*, on reproche aussi à la méthode le fait de donner une importance disproportionnée à la variable encodée (une partie significative des *features* ne concerne qu'une seule variable) avec un possible impact sur la capacité du modèle à généraliser. 

Remarque: Dans le cas des variables ordinales, l'encoding le plus simple est de mapper les $n$ catégories sur l'ensemble des $n$ premiers entiers naturels. 

Parmi les *encoders* classiques, on trouve en plus du *one-hot encoding*:
* Le *binary encoding* commence par effectuer un *ordinal encoding* qui associe à chaque catégorie un nombre entier qui est ensuite converti en binaire ($p$ bits). Le nombre de nouvelles variables correspond aux nombre de bits ce qui peut être substantiellement inférieur au nombre obtenu par *one-hot encoding* quand la cardinalité s'élève. Par exemple: pour 16 catégories, le *one-hot encoder* génère 15 colonnes, un *4-binary encoder* n'en génère seulement que 4. 
* Le *hashing encoder* est similaire au *binary encoder* et s'appuie notamment sur la propriété qu'une fonction de hachage fait d'un input de n'importe quelle longueur un output de longueur fixe (le *hashing trick*) qui est finalement encodé (en général sur 8 bits par défait) avec le risque de perte d'information lié aux possibles collisions.

Les *encoders* classiques ont le principal inconvénient de n'inclure que le minimum d'information sur la variable encodée, en particulier, les *encodings* classiques d'une variable catégorielles n'incluent aucune information sur sa relation avec la réponse et/ou avec les autres variables du problème. Par exemple: un *encoding* classique n'apporte rien d'autre que la fréquence relative d'apparition de chaque catégories dans le *dataset*, il ne permet par exemple pas de dire que deux catégories jouent le même rôle du point de vue de notre problème. Une catégorie d'*encoders* dits bayésiens vient adresser le premier point, le second étant pris en charge par les plus sophistiqués *embeddings*.

### *Encoders bayésiens*
Derrière le terme sophistiqué de "bayésien" se cache juste l'idée de vouloir inclure dans l'*encoding* des informations quand à la relation entre notre variable dépendante/cible/réponse (*target*) et la variable catégorielle à encoder. L'avantage d'inclure de telles information dans l'*encoding* est par exemple en classification, d'obtenir une nouvelle variable préservant ou au moins contribuant à la séparabilité du problème.

Le plus simple de ces *encoders* s'appelle le *target encoder* qui consiste à créer une nouvelle variable où chaque catégorie est encodée par la valeur moyenne de la réponse sur la catégorie. On peut finalement envisager l'utilisation de n'importe quelle fonction d'aggrégation autre que la moyenne. Par exemple, une fonction d'aggrégation classique utilisée en classification pour le *target encoding* est le *supervised ratio* qui correspond à la proportion de 1 dans la catégorie.

Remarque: Du point de vue de la dimensionalité, ces techniques on l'avantage par rapport aux *encodings* classiques de ne créer qu'une seule nouvelle *feature*.

On trouve aussi le *leave-one-out encoding* comme  variante du *target encoding*, le principe est le même le calcul de l'aggrégat pour une catégorie donnée se fait sur l'ensemble du *dataset* moins cette catégorie contre l'ensemble du *dataset* pour le target encoding.

De tels *encodings* produisent de l'information sur les catégories obtenues dans le sens où deux catégories encodées avec des valeurs proches jouent un rôle similaire du point de vue de la réponse. Inversement, si cela donne des résultats en contradiction avec des logiques métiers, mieux vaut reévaluer son choix dans la méthode d'*encoding*. Par exemple, le *frequency embedding* encode la variable avec sa fréquence d'apparition dans le *dataset*, ce qui fait implicitement l'hypothèse qu'on considère deux catégories de même fréquence comme semblables du point de vue de leur effet sur la réponse.

Parmi les *encoders* bayésiens, on trouve également: James-Stein, M-Estimator ou encore Weights of Evidence.

### Autres *encoders*
On trouve notamment ce qu'on appelle les *contrast encoders* semblant être dédiés aux variables ordinales mais dont l'utilisation semble délicate.

Voir notamment ce [lien](https://stats.idre.ucla.edu/r/library/r-library-contrast-coding-systems-for-categorical-variables/) au sujet des *contrast encoder* et ce [notebook](https://github.com/FeatureLabs/categorical_encoding/blob/master/guides/notebooks/categorical-encoding-guide.ipynb) pour une vue d'ensemble des *encoders* abordés ici.

### Packages
`sklearn` à travers sa librairie `sklearn.preprocessing` ne propose que les encoders de base (`OneHotEncoder` et `OrdinalEncoder`). Voir surtout le projet [`categorical-encoding`](https://github.com/scikit-learn-contrib/categorical-encoding) qui fait partie comme par exemple `imbalance-learn`, d'un groupe de projets compatibles avec et soutenus par `sklearn` (et regroupées dans ce repo [git](https://github.com/scikit-learn-contrib))

## *Embeddings*
L'idée des *embeddings* est de capturer les relations qu'a une variables catégorielle avec la variable dépendante/réponse et/ou d'autres variables du problème dans une représentation, un vecteur numérique de faible dimension (on parle aussi de Cat2Vec). Il y a une part de réduction de dimension dans la technique au sens où on checher à ce que la dimension de l'*embedding* soit beaucoup plus faible que la cardinalité de la variable traitée. On perd toutefois en interprétabilité mais on gagne souvent en généralisation notamment dans le cas de *datasets* avec des variables catégorielles à cardinalité élevée où les données sont très *sparses* et où des algorithmes auraient tendance à *overfitter* avec d'autres techniques. 

Remarque: Les *encodings* bayésiens constituent de ce point de vue les plus simples des *embeddings* (où la dimension de l'*embedding* est 1). 

Un *embedding* se construit avec un réseau de neurones. On commence par un *ordinal encoding* de la variable catégorielle pour pouvoir passer des valeurs numériques au réseau. L'input est passé à une couche appelée couche d'*embedding* (dont on récupérera les poids une fois le réseau entrainé pour calculer les vecteurs de l'*embedding*) qui est ensuite suivie de quelques couches denses avant d'arriver à la couche de sortie (il est sous-entendu que l'*embedding* est construit de façon supervisée).

On peut construire les *embeddings* de plusieurs variables catégorielles en même temps. Cela permet notamment de capturer les relations qu'elles peuvent avoir entre elles et avec la réponse. On ajoutera alors une couche concaténant les différents embeddings avant que ceux-ci ne soient passés aux couches denses. On peut à ce stade également passer aux couches denses d'autres variables numériques du problème dont on pense qu'il serait utile de capturer dans les différents *embeddings* leur relation avec les différentes variables catégorielles.

Remarque: Il est nécessaire de choisir une taille pour chacune des couches d'*embedding* (taille du vecteur/dimension du sous-espace des *embeddings* appelé aussi *latent space*). Il existe des règles type $min(50, cardinality/2)$ (qui peuvent ne pas économiser un *grid search*) mais globalement le choix de la dimension du *latent space* résulte d'un classique compromis biais/variance: si prise trop faible, on *underfit*, on perd trop d'information ; si trop élevée, on risque d'overfitter/d'apprendre le *dataset*.

## Autres techniques
On mentionne ici l'équivalent de la PCA pour les variables catégorielles : la MCA (*Multiple Correspondence Analysis*). Quel est son apport ? Est-elle adaptée à l'*encoding* de variables catégorielles ?